In [ ]:
https://docs.osmcode.org/pyosmium/latest/ref_index.html
https://osmcode.org/osmium-tool/manual.html
https://docs.osmcode.org/osmium/latest/

In [7]:
import osmium as osm
import pandas as pd
from overpass import API

api = API()


class OSMHandler(osm.SimpleHandler):
    
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.address = {'addr:housenumber': '2400', 'addr:postcode': '98144', 'addr:street': 'South Hill Street'}
        self.roads_with_latlon = []

    def node(self, n):
        dict = {}
        the_node = 0
        for tag in n.tags:
            dict[tag.k] = tag.v
        for key in self.address:
            if key not in dict:
                the_node = 1
                break
            if dict[key] != self.address[key]:
                the_node = 1
                break
        if the_node == 0:
            self.lat = n.location.lat
            self.lon = n.location.lon
            response = api.get('way["highway" ="residential"](around:95,{0},{1});'.format(n.location.lat,n.location.lon)) 
            # print('For location with coordinates {0}, {1} found way IDs:'.format(loc[0],loc[1]))
            self.roads = []
            print(response.features)
            for way in response.features:
                self.roads.append(way.id)

    def way(self, w):
       
        if w.id in self.roads:
            list = []
            # tags = []
            for road in w.nodes:
                list.append(road)
            # for tag in w.tags:
            #     tags.append(tag)
            # self.roads_with_latlon.append([list, tags])
            self.roads_with_latlon.append(list)
         # self.osm_data[w.id] =  [nodeList, tagList]
        
    # def relation(self, r):
    #     self.tag_inventory(r, "relation")

    
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("Downloads/map.osm", locations=True, idx='flex_mem')

# transform the list into a pandas DataFrame
# data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
#                  'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
# df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
# osmhandler.roads_with_latlon


[{"geometry": {"coordinates": [], "type": "LineString"}, "id": 6487472, "properties": {"bicycle": "designated", "cycleway": "shared_lane", "highway": "residential", "name": "24th Avenue South", "tiger:cfcc": "A41", "tiger:county": "King, WA", "tiger:name_base": "24th", "tiger:name_direction_suffix": "S", "tiger:name_type": "Ave", "tiger:reviewed": "no"}, "type": "Feature"}, {"geometry": {"coordinates": [], "type": "LineString"}, "id": 481289523, "properties": {"highway": "residential", "name": "24th Avenue South", "surface": "asphalt", "tiger:cfcc": "A41", "tiger:county": "King, WA", "tiger:name_base": "24th", "tiger:name_direction_suffix": "S", "tiger:name_type": "Ave", "tiger:reviewed": "no"}, "type": "Feature"}, {"geometry": {"coordinates": [], "type": "LineString"}, "id": 481289524, "properties": {"highway": "residential", "name": "South Hill Street", "tiger:cfcc": "A41", "tiger:county": "King, WA", "tiger:name_base": "Hill", "tiger:name_direction_prefix": "S", "tiger:name_type": "

In [8]:
osmhandler.roads_with_latlon

[[osmium.osm.NodeRef(ref=53136415, location=osmium.osm.Location(x=-1223015067, y=475869198)),
  osmium.osm.NodeRef(ref=5757431543, location=osmium.osm.Location(x=-1223015731, y=475868612)),
  osmium.osm.NodeRef(ref=2516369689, location=osmium.osm.Location(x=-1223015896, y=475868466)),
  osmium.osm.NodeRef(ref=152010665, location=osmium.osm.Location(x=-1223016159, y=475868093)),
  osmium.osm.NodeRef(ref=2516441880, location=osmium.osm.Location(x=-1223016311, y=475867646)),
  osmium.osm.NodeRef(ref=9193214107, location=osmium.osm.Location(x=-1223016316, y=475866142)),
  osmium.osm.NodeRef(ref=5440359896, location=osmium.osm.Location(x=-1223016323, y=475864945)),
  osmium.osm.NodeRef(ref=4799960441, location=osmium.osm.Location(x=-1223016328, y=475863911)),
  osmium.osm.NodeRef(ref=6632334769, location=osmium.osm.Location(x=-1223016329, y=475863758)),
  osmium.osm.NodeRef(ref=53138092, location=osmium.osm.Location(x=-1223016332, y=475863228)),
  osmium.osm.NodeRef(ref=4799960440, location

In [19]:
min_of_closest_points = []
minimum = float('inf')
for road in osmhandler.roads_with_latlon: #4 elements
    y_distance = road[1][0].location.lat - osmhandler.lat
    x_distance = road[1][0].location.lon - osmhandler.lon
    total = abs(x_distance + y_distance)
        
    minimum = min(total, minimum)
        
    min_of_closest_points.append([minimum,road[0], road[1][0]])


lowest = float('inf')

for point in min_of_closest_points:
    if lowest > point[0]:
        lowest = point[0]
        road = point

print(road)

[0.0006808999999918797, 481289523, osmium.osm.NodeRef(ref=53232306, location=osmium.osm.Location(x=-1223016580, y=475845040))]


In [9]:
class RoadNode:
    def __init__(self, r):
        self.visited = 0
        self.nodes = []
        self.intersection = 0
        if r != None:
            self.ref = r.ref
            self.location = r.location
        else:
            self.ref = None
            self.location = None
    def addNode(self, node):
        self.nodes.append(node)
    
    def getVisit(self):
        return self.visited

        


#if a road is connected (meaning one road contains a ref of another road), the road is now one big road consisting of the smaller roads

#the algo calculates routes at each point then pick the route with the shortest time to complete
road_nodes = {}
prev_node = RoadNode(r = None)

for roadds in osmhandler.roads_with_latlon:
    for road in roadds:
        #inserting to road_nodes dict.
        if road.ref in road_nodes:
            if prev_node.ref != road.ref:
                road_node = road_nodes[road.ref]
                prev_node = road_node
            # if prev_node.ref != None and prev_node.ref != road.ref:    
            
        else:
            node = RoadNode(road)            
            road_nodes[road.ref] = node
            node.addNode(prev_node)
            prev_node.addNode(node)
            prev_node = node


for key in road_nodes:
    if len(road_nodes[key].nodes) > 2:
        road_nodes[key].intersection = 1



        

In [14]:
sum = 0
for ro in osmhandler.roads_with_latlon:
    print(len(ro))
    sum = sum + len(ro)
print(sum)

19
4
5
6
17
51


In [11]:
def fly_drone_to_dest(start, dest):
    



def instructions_dfs(roadNode, visited, drone_pos):
    skip = 0
    shortest_time = [len(visited), visited, drone_pos]

    
    
    roadNode.visited = 1
    for r in roadNode.nodes:
        if r in visited:
            skip = skip + 1
            continue
       
        visited = visited + [r]

        shortest_time = instructions_dfs(r, visited, drone_pos)

    if skip == len(roadNode.nodes) drone_pos = roadNode
    return shortest_time
    
all_dir = []
visited = []
drone_pos = None
for key in road_nodes: 
    #instructions_dfs return [time, dir_of_cur_start]
    if road_nodes[key].visited == 1: 
        continue
    visited = visited + [road_nodes[key]]
    drone_pos = road_nodes[key]
    list = instructions_dfs(road_nodes[key], visited, drone_pos)
    all_dir.append(list)

all_dir
    
    

[[27,